In [6]:
import numpy as np
import pandas as pd
from itertools import combinations
import random

data = pd.read_csv(r'dados/movie_metadata.csv')
budgets = [0.1, 1, 10]

data.drop_duplicates()
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


# Consultas

In [2]:
# Consulta 1: Filme com a maior arrecadação
def consulta1(data):
    aux = data[['movie_title', 'gross'][:]].dropna()
    return aux.loc[aux.gross.idxmax()]

# Consulta 2: Filme com maior arrecadação para cada idioma
def consulta2(data):
    aux = data[['movie_title', 'language', 'gross'][:]].dropna()
    return aux.loc[aux.groupby('language').gross.idxmax()]

# Consulta 3: Top 3 países com a maior quantidade de filmes
def consulta3(data):
    result = []
    aux = data[['country', 'movie_title'][:]].dropna()
    country_list = aux.groupby('country')['movie_title'].nunique().to_frame()
    country_list['country'] = country_list.index # o índice se perde ao agrupar
    country_list.reset_index(drop=True, inplace=True)
    for i in range(3):
        result.append(country_list.loc[country_list.movie_title.idxmax()])
        country_list = country_list.drop(country_list.movie_title.idxmax())
    return pd.DataFrame(result)

# Funções Score

In [3]:
def score1(item, out):
    score = 0
    if(not np.isnan(item.gross) and item.movie_title != 'None'):
        if(item.movie_title == out):
            score = item.gross
    return score

def score2(item, out):
    score = 0
    if(item.movie_title in out and not np.isnan(item.gross)):
        score += item.gross
    return score

def score3(data, item, out):
    score = 0
    total_countries = data.country.array
    country_list = {}
    for i in range(0, len(total_countries)):
        if total_countries[i] not in country_list:
            country_list[total_countries[i]] = 0
            
    for i in range(0, len(data)):
        if type(data.iloc[i].country != float):
            country_list[data.iloc[i].country] += 1
    if item.country in out:
        score = country_list[item.country]
    return score

# Funções de sensibilidade

In [4]:
def sense_q1(data):
    mat = []
    sense = 0
    results = data[['movie_title'][:]]
    for i in range(0, len(results)):
        mat.append(score1(data.iloc[i], results.iloc[i].movie_title))
        
    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

def sense_q2(data):
    mat = []
    sense = 0
    results = data[['movie_title', 'language', 'gross'][:]]
    for i in range(0, len(results)):
        mat.append(score2(results.iloc[i], data.iloc[i].movie_title))
    
    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

def sense_q3(data):
    sense = 0
    country_list = []
    mat = []
    for i in range(0, len(data)):
        if(data.iloc[i].country not in country_list and data.iloc[i].country != 'None'):
            country_list.append(data.iloc[i].country)
    results = list(combinations(country_list, 3))
    
    for r in range(0, len(results)):
        mat.append(score3(data, data.iloc[r], results[r]))

    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

596662957.0

# Mecanismo Exponencial

# Randomização de consultas

In [23]:
def rand_q1(data):
    aux = data[['movie_title', 'gross'][:]].dropna()
    return random.choice(aux.movie_title).replace(u'\xa0', u'')

def rand_q2(data):
    return

def rand_q3(data):
    return

rand_q1(data)

'The Smurfs'

# Execução

In [5]:
# TODO